# Setup

Initial module setup.

In [1]:
import logging
import sys
import numpy.typing as np_types 
import pandas as pd

from xgboost import XGBClassifier
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion, logging_helpers

# Constants

In [2]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10
RANDOM_SEED = 42

# Log Output

In [3]:
logger = logging.getLogger(logging_helpers.LOGGER_NAME)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [4]:
class XGBBuilder(model.ModelBuilder[XGBClassifier]):    
    def create_classifier(self) -> XGBClassifier:
        # see: https://github.com/Rahul2K20/EEG_Authentication/blob/main/Main.ipynb
        return XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.3,
            random_state=RANDOM_SEED
        )
    
    def train_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [5]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter_filter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)
converter_psd = conversion.MNEDataFrameConverter(sample_frequency=DATASET_SAMPLE_FREQ_HZ)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [6]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        converter_filter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [7]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.PSDExtractor(converter_psd)
])

## Data Processor

In [8]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [9]:
xgb_builder = XGBBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = xgb_builder.train(K_FOLDS)

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
  

In [10]:
subject_score_rows = []
for subject in results.training_statistics:
    subject_score_rows.append([
        subject,
        results.training_statistics[subject].average_score,
        results.training_statistics[subject].training_duration
    ])
subject_scores = pd.DataFrame(subject_score_rows, columns=['Subject', 'Average Score', 'Training Duration'])
subject_scores

,Subject,Average Score,Training Duration
0,S01,0.972731,41.564002
1,S02,0.988827,47.620002
2,S03,0.987546,40.413499
3,S04,0.972701,41.991000
4,S05,0.985170,44.803000
5,S06,0.941806,48.717499
6,S07,0.975216,42.076501
7,S08,0.983873,36.669501
8,S09,0.998750,34.076002
9,S10,0.986343,38.773003


In [11]:
subject_rates_rows = []
for subject in results.training_statistics:
    subject_rates_rows.append([
        subject,
        results.training_statistics[subject].average_false_accept_rate,
        results.training_statistics[subject].average_false_reject_rate
    ])
subject_rates = pd.DataFrame(subject_rates_rows, columns=['Subject', 'Average FAR', 'Average FRR'])
subject_rates

,Subject,Average FAR,Average FRR
0,S01,0.009125,0.375000
1,S02,0.001299,0.208333
2,S03,0.002597,0.216667
3,S04,0.002632,0.500000
4,S05,0.006721,0.109524
5,S06,0.025974,0.700000
6,S07,0.003896,0.425000
7,S08,0.001299,0.308333
8,S09,0.000000,0.033333
9,S10,0.001316,0.250000


In [12]:
global_stats = pd.DataFrame(
    [[results.global_average_score, results.global_average_far_rate, 
      results.global_average_frr_rate, results.global_average_time]],
    columns=['Global Average Score', 'Global Average FAR', 'Global Average FRR', 'Global Average Time']
)
global_stats

,Global Average Score,Global Average FAR,Global Average FRR,Global Average Time
0,0.97674,0.21381,0.013232,40.565901
